# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [1]:
!pip install folium
!pip install scikit-optimize

     |████████████████████████████████| 94 kB 4.3 MB/s eta 0:00:011
     |████████████████████████████████| 101 kB 5.1 MB/s ta 0:00:011


In [2]:
import pandas as pd
import numpy as np
import folium


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [4]:
# install, in cell 1 already installed
#!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [5]:
from skopt import BayesSearchCV as BSCV
from skopt.space.space import Integer as skopt_int
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.svm import SVR 
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [6]:
np.shape(X)

(400000, 31)

In [7]:
y_less = y[:10000]
X_less=X[:10000]
print(np.shape(X_less), np.shape(y_less))

(10000, 31) (10000,)


In [8]:
X_l_train,X_l_test,y_l_train,y_l_test = train_test_split(X_less,y_less, test_size=0.1, random_state = 1)

In [9]:
print(type(y_l_train), type(y_l_test))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


### ***Linear regression***
- There are no parameters to be optimzied. So no BayesSearchCV is used

In [10]:
LRm = LR().fit(X_l_train,y_l_train)

In [11]:
res_LR_y_l_test = LRm.predict(X_l_test)

In [12]:
mse_LR_test = mse(y_l_test,res_LR_y_l_test)
mse_LR_test

25.88405237662758

### ***Random Forest Tree***

In [13]:
RFR_pl = make_pipeline(StandardScaler(),RFR(random_state = 0))

In [14]:
RFR_pl.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'randomforestregressor', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'randomforestregressor__bootstrap', 'randomforestregressor__ccp_alpha', 'randomforestregressor__criterion', 'randomforestregressor__max_depth', 'randomforestregressor__max_features', 'randomforestregressor__max_leaf_nodes', 'randomforestregressor__max_samples', 'randomforestregressor__min_impurity_decrease', 'randomforestregressor__min_impurity_split', 'randomforestregressor__min_samples_leaf', 'randomforestregressor__min_samples_split', 'randomforestregressor__min_weight_fraction_leaf', 'randomforestregressor__n_estimators', 'randomforestregressor__n_jobs', 'randomforestregressor__oob_score', 'randomforestregressor__random_state', 'randomforestregressor__verbose', 'randomforestregressor__warm_start'])

In [15]:
search_spaces = {'randomforestregressor__n_estimators': skopt_int(1,1000),
                  'randomforestregressor__max_depth' : skopt_int(2,100)}

In [55]:
RFR_BSCV = BSCV(estimator = RFR_pl, search_spaces = search_spaces, n_iter=50,n_jobs=-1,random_state=0)

In [ ]:
RFR_BSCV.fit(X_l_train,y_l_train)

/home/student/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
RFR_BSCV.best_estimator_

In [ ]:
#RFR_BSCV.cv_results_

In [ ]:
RFR_BSCV.best_estimator_.get_params()['randomforestregressor']

In [ ]:
RFR_BSCV.best_score_

n_iter=30,
best_estimator_params,
- max_depth = 27
- n_estimators = 979
- best_score_ = 0.77707

n_iter=50,
best_estimator_params,
- max_depth = 
- n_estimators = 
- best_score_ = 


### ***SVM***

In [ ]:
from skopt.space.space import Categorical as skopt_cat
from skopt.space.space import Real as skopt_real

In [ ]:
SVR_pl = make_pipeline(StandardScaler(),SVR(kernel='rbf'))

In [ ]:
SVR_pl.get_params().keys()

In [ ]:
search_spaces = {'svr__gamma': skopt_cat(['scale','auto']),
                  'svr__C' : skopt_real(0.01,100)}

In [ ]:
SVR_BSCV = BSCV(estimator = SVR_pl, search_spaces = search_spaces, n_iter=50,n_jobs=-1,random_state=0)

In [ ]:
SVR_BSCV.fit(X_l_train,y_l_train)

In [ ]:
SVR_BSCV.best_estimator_

In [ ]:
#SVR_BSCV.cv_results_

In [ ]:
#SVR_BSCV.best_estimator_.get_params()['??']

In [ ]:
SVR_BSCV.best_score_

n_iter=50,
best_estimator_params,
- gamma=
- C = 
- best score = 